In [0]:
%run ../_config

In [0]:
from pyspark.sql.functions import col, count, countDistinct

# 1. Infrastructure: Create the Gold Schema
# We use Python-wrapped SQL to ensure the schema exists before writing.
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalog}.{gold_schema}")

# 2. Read from Silver (The Source of Truth)
# We read the Silver table as a Batch because Gold is an aggregate.
silver_df = spark.read.table(f"{catalog}.{silver_schema}.silver_clickstream")

# --- TABLE 1: Daily Active Users (DAU) ---
# Purpose: Simple business health metric.
dau_df = (
    silver_df
    .groupBy("year", "month", "day")
    .agg(countDistinct("user_id_hashed").alias("unique_users"))
    .orderBy("year", "month", "day", ascending=False)
)

# Write to Gold (Overwrite mode to refresh the full daily calculation)
(dau_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(f"{catalog}.{gold_schema}.daily_active_users"))

# --- TABLE 2: Traffic Performance (Master Gold) ---
# Purpose: Allows "Slicing and Dicing" by country, source, and device.
traffic_df = (
    silver_df
    .groupBy("year", "month", "day", "country", "utm_source", "browser")
    .agg(
        countDistinct("user_id_hashed").alias("unique_users"),
        count("event_id").alias("total_clicks")
    )
    .withColumn("engagement_score", col("total_clicks") / col("unique_users"))
)

# Write to Gold (Overwrite mode with mergeSchema=True for resilience)
(traffic_df.write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable(f"{catalog}.{gold_schema}.traffic_performance"))

print(f"Gold tables successfully created in {catalog}.{gold_schema}")

# 3. Quick Preview of results
display(spark.read.table(f"{catalog}.{gold_schema}.traffic_performance").limit(10))